In [2]:
# import designMethods.en_13001_3_3 as en
# from output import create_output_file
from IPython.display import display, FileLink
import ipywidgets as widgets
import pathlib
import os
import itertools
from marsDemonstrator import Main_application

In [4]:
# !pip install voila
# !jupyter serverextension enable voila --sys-prefix

In [6]:
# en_13001 = {}
# gen_vars = {"num_run": 1, "input_file": None, "output_file": None}
# direction = 1
en_13001 = Main_application()


In [8]:
def read_file(input_file):
    en_13001["input"].load_gp_input(input_file, "configuration")

    # load material and en 13001 parameters
    en_13001["input"].load_material_input_check(input_file, "rail_materials", "wheel_materials")
    en_13001["input"].load_parameter_input(input_file, "Input_variables")

    # check for input errors and drop faulty configurations
    en_13001["input"].perform_error_checks()
    en_13001["input"].drop_error_configs()

    # precompute factors and material parameters
    en_13001["input"].parameters.compute_f_f3()
    en_13001["input"].parameters.compute_contact_and_f_1()
    en_13001["input"].set_materials_and_geometry()

In [10]:
def init_gps(change):
    msg_box.value = "Initializing GPs: this may take up to 30 seconds"
    en_13001.input.config = drop_config.value
    try:
        en_13001.init_gps()
    except Exception as e:
        msg_box.value(e)
    en_13001.config_loaded = True
    # # initialize prediction class
    # en_13001["computed"] = en.Computed_data()

    # # get current config (1 mast or 2 mast)
    # en_13001["input"].config = drop_config.value

    # # 1 mast configs have to sub configs: cg on the left (direction = 1) or right (direction = -1)
    # gp_configs = {"m1": {1: "m1l", -1: "m1r"}, "m2": {1: "m2"}}
    # gp_config = gp_configs[en_13001["input"].config][direction]

    # # parts = ["wf", "wr", "r"]
    # # en_13001["computed"].get_gps_kc(gp_config, parts)

    # en_13001["computed"].load_gps(gp_config)
    # en_13001["input"].config_loaded = True
    msg_box.value = "Initialized GPs"

In [12]:
def start_computation(change):

    # if there were never inputs loaded throw error message 
    if en_13001.input_file_loaded is None:
        if uploader._counter == 0:
            msg_box.value = "Please upload an excel file"
            return
    
    # if there is a new upload file load its content and do make new computation
    if uploader._counter > 0:
        # initialize input class
        en_13001.input = en.EN_input()

        # file to temporarily save input data to
        en_13001.input_file_path = "input.xlsx"
        with open(en_13001.input_file_path, "w+b") as i:
            i.write(uploader.data[-1])

        # read temp file and delete after

        en_13001.read_file(en_13001.input_file_path)
        os.remove(en_13001.input_file_path)

        # reset uploader after reading file
        uploader.value.clear()
        uploader._counter = 0
        
        # if gps for configuration were not loaded yet load them
        if en_13001.input_file_loaded is None:
            en_13001.init_gps()
        msg_box.value = "Start Computation"

        en_13001.run_computation_and_create_output(1)

        # # recompute gp data --> normalization
        # en_13001.input.recompute_gp_data(en_13001.inputconfig)

        # # make predictions for k_c and travelled distance (needed for i_tot)
        # en_13001.prediction.predict_kc(en_13001.input.gp_input.norm)
        # en_13001.prediction.compute_F_sd_f_all(en_13001.input.gp_input.raw, en_13001.input.config, direction)
        # en_13001.prediction.predict_travelled_dist(en_13001.input.parameters.data)

        # # create computation instance and compute all sc configuration
        # en_13001.computation = en.Computation(en_13001.input, en_13001.prediction)
        # en_13001.computation.compute_pre_F_rd_all()
        # en_13001.computation.compute_F_rd_all()
        # en_13001.computation.compute_proofs_all()

        # # create result table
        # en_13001.input.prepare_for_output()
        # en_13001.computation.load_results_all()

    msg_box.value = "Finished computation"

    # create output file
    # gen_vars["output_file"] = "./output.xlsx"
    # create_output_file(en_13001["computation"], en_13001["input"], gen_vars["output_file"])
    local_file = FileLink(en_13001.outname, result_html_prefix="Click here to download results: ")

    # display output file and write errors to output if there are any
    out_file.clear_output()
    out_errors.clear_output()
    with out_file:
        display(local_file)
        display(btn_delete_output)
    with out_errors:
        for error in list(itertools.chain(*en_13001["input"].error_report)):
            print(error)

In [14]:
def delete_output_file(change):
    os.remove(gen_vars["output_file"])
    out_file.clear_output()

In [16]:
## Create widtges
# uploader for input
uploader = widgets.FileUpload(accept=".xlsx", multiple=False)

# dropdown for computation mode
drop_mode = widgets.Dropdown(description="Computation Mode", options=["proof", "min_diameter"], value="proof")

# dropdown for configuration
drop_config = widgets.Dropdown(description="SC Configuration", options=["m1", "m2"], value="m1")

# button for updating configuration
btn_update_config = widgets.Button(description="Update Configuration")
btn_update_config.on_click(init_gps)

# button to start computation
btn_start = widgets.Button(description="Start computation")
btn_start.on_click(start_computation)

# output widget to display output file
out_file = widgets.Output()

# output widget to write error reports
out_errors = widgets.Output()

# message box for information on computation process
msg_box = widgets.Text(value="", description="Message Box")

# button to delete output file from server
btn_delete_output = widgets.Button(description="Delete Resultfile")
btn_delete_output.on_click(delete_output_file)

In [18]:
## Build gui
widgets.VBox([widgets.Label("MARS Demonstrator"),
              drop_mode, drop_config, btn_update_config, uploader, msg_box, btn_start, out_file, out_errors])

In [20]:
# en_13001.init_gps()

In [22]:
en_13001.input.config